In [3]:
import numpy as np
import pandas as pd
import cv2
import skimage
from PIL import Image
from ultralytics import YOLO
from sklearn.metrics import mean_squared_error
import torch
import json
import yaml
import time
from scipy.spatial import distance as sci_dist
import matplotlib.pyplot as plt

#  matplotlib inline is set 
%matplotlib inline

# Get tactical map keypoints positions dictionary
json_path = "./pitch map labels position.json"
with open(json_path, 'r') as f:
    keypoints_map_pos = json.load(f)

# Get football field keypoints numerical to alphabetical mapping
yaml_path = "./config/pitch_dataset.yaml"
with open(yaml_path, 'r') as file:
    classes_names_dic = yaml.safe_load(file)
classes_names_dic = classes_names_dic['names']

# Get football field keypoints numerical to alphabetical mapping
yaml_path = "./config/players_dataset.yaml"
with open(yaml_path, 'r') as file:
    labels_dic = yaml.safe_load(file)
labels_dic = labels_dic['names']

# Set video path
video_path = './demo_vid_1.mp4'

# Read tactical map image
tac_map = cv2.imread('./tactical map.jpg')

# Define team colors (based on chosen video)
nbr_team_colors = 2
colors_dic = {
    "France": [(64, 224, 208), (144, 238, 144)],  # France colors (Players kit color in light turquoise, GK kit color in light green)
    "Switzerland": [(0, 0, 0), (255, 255, 0)]  # Switzerland colors (Players kit color in black, GK kit color in yellow)
}

colors_list = colors_dic["France"] + colors_dic["Switzerland"]  # Define color list to be used for detected player team prediction
color_list_lab = [skimage.color.rgb2lab([i / 255 for i in c]) for c in colors_list]  # Converting color_list to L*a*b* space

# Load the YOLOv8 players detection model
model_players = YOLO("./models/Yolo8L_Players/weights/best.pt")

# Load the YOLOv8 field keypoints detection model
model_keypoints = YOLO("./models/Yolo8M_Field_Keypoints/weights/best.pt")

# Load the YOLOv5 goal detection model using torch.hub
yolov5_path = 'yolov5'  # Adjust the path if necessary
goal_model_path = 'models/goals.pt'
model_goal = torch.hub.load(yolov5_path, 'custom', path=goal_model_path, source='local')

# Open video file
cap = cv2.VideoCapture(video_path)

# Initialize frame counter
frame_nbr = 0

# Set keypoints average displacement tolerance level (in pixels) [set to -1 to always update homography matrix]
keypoints_displacement_mean_tol = 10

# Set confidence thresholds for players, field keypoints, and goal detections
player_model_conf_thresh = 0.60
keypoints_model_conf_thresh = 0.70
goal_model_conf_thresh = 0.60

# Set variable to record the time when we processed last frame
prev_frame_time = 0
# Set variable to record the time at which we processed current frame
new_frame_time = 0

# Store the ball track history
ball_track_history = {'src': [], 'dst': []}

# Count consecutive frames with no ball detected
nbr_frames_no_ball = 0
# Threshold for number of frames with no ball to reset ball track (frames)
nbr_frames_no_ball_thresh = 30
# Distance threshold for ball tracking (pixels)
ball_track_dist_thresh = 100
# Maximum ball track length (detections)
max_track_length = 35

# Ball position
bola_x = 0
bola_y = 0

# Index of the player with the ball
jogador_com_bola = False
equipa_com_bola = ""
equipa_anterior_com_bola = ""
ultima_accao = ""

# Possible actions
passe = False
interceccao = False
recepcao = False
goal_try = False
goal = False

encontrou_jogador_com_bola = False
dist_euclideana = -1

min_dist_jogador_bola = 100000000

# Heuristic X: goal attempt: team makes a "pass" to the opposing goal area

# Thresholds - Control radius for ball possession
threshold_posse_bola = 25
goal_try_threshold = 100  # Distance threshold for goal try

detected_labels_prev = []
prev_ball_pos = None  # Variable to store previous ball position

# Initialize the video writer outside the loop
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec definition
output_path = 'output.mp4'
frame_rate = 5.0  # Adjust as necessary
out = None

# Initialize detected_ball_dst_pos
detected_ball_dst_pos = None

# Function to find the nearest keypoint to a given point
def find_nearest_keypoint(point, keypoints):
    distances = [np.linalg.norm(point - kp) for kp in keypoints]
    nearest_keypoint_idx = np.argmin(distances)
    return keypoints[nearest_keypoint_idx]

# Function to find the two nearest keypoints to a given point
def find_two_nearest_keypoints(point, keypoints):
    distances = [np.linalg.norm(point - kp) for kp in keypoints]
    nearest_keypoint_idxs = np.argsort(distances)[:2]
    return keypoints[nearest_keypoint_idxs[0]], keypoints[nearest_keypoint_idxs[1]]

# Loop through the video frames
while cap.isOpened():
    # Update frame counter
    frame_nbr += 1

    # Read a frame from the video
    success, frame = cap.read()

    # Skip frames to handle computer limitations
    if frame_nbr % 5 != 0:
        continue

    # Reset tactical map image for each new frame
    tac_map_copy = tac_map.copy()

    # Reset ball tracks
    if nbr_frames_no_ball > nbr_frames_no_ball_thresh:
        ball_track_history['dst'] = []
        ball_track_history['src'] = []

    # Process the frame if it was successfully read
    if success:
        #################### Part 1 ####################
        # Object Detection & Coordinate Transformation #
        ################################################

        # Run YOLOv8 players inference on the frame
        results_players = model_players(frame, conf=player_model_conf_thresh)
        # Run YOLOv8 field keypoints inference on the frame
        results_keypoints = model_keypoints(frame, conf=keypoints_model_conf_thresh)
        # Run YOLOv5 goal detection model on the frame
        results_goal = model_goal(frame)

        ## Extract detections information
        bboxes_p = results_players[0].boxes.xyxy.cpu().numpy()  # Detected players, referees and ball (x,y,x,y) bounding boxes
        bboxes_p_c = results_players[0].boxes.xywh.cpu().numpy()  # Detected players, referees and ball (x,y,w,h) bounding boxes
        labels_p = list(results_players[0].boxes.cls.cpu().numpy())  # Detected players, referees and ball labels list
        confs_p = list(results_players[0].boxes.conf.cpu().numpy())  # Detected players, referees and ball confidence level

        bboxes_k = results_keypoints[0].boxes.xyxy.cpu().numpy()  # Detected field keypoints (x,y,w,h) bounding boxes
        bboxes_k_c = results_keypoints[0].boxes.xywh.cpu().numpy()  # Detected field keypoints (x,y,w,h) bounding boxes
        labels_k = list(results_keypoints[0].boxes.cls.cpu().numpy())  # Detected field keypoints labels list

        bboxes_g = results_goal.xyxy[0].cpu().numpy()  # Detected goals (x,y,x,y) bounding boxes
        confs_g = results_goal.xyxy[0][:, 4].cpu().numpy()  # Detected goal confidence levels

        # Convert detected numerical labels to alphabetical labels
        detected_labels = [classes_names_dic[i] for i in labels_k]

        # Extract detected field keypoints coordinates on the current frame
        detected_labels_src_pts = np.array([list(np.round(bboxes_k_c[i][:2]).astype(int)) for i in range(bboxes_k_c.shape[0])])

        # Get the detected field keypoints coordinates on the tactical map
        detected_labels_dst_pts = np.array([keypoints_map_pos[i] for i in detected_labels])

        ## Calculate Homography transformation matrix when more than 4 keypoints are detected
        if len(detected_labels) > 3:
            # Always calculate homography matrix on the first frame
            if frame_nbr > 1:
                # Determine common detected field keypoints between previous and current frames
                common_labels = set(detected_labels_prev) & set(detected_labels)
                # When at least 4 common keypoints are detected, determine if they are displaced on average beyond a certain tolerance level
                if len(common_labels) > 3:
                    common_label_idx_prev = [detected_labels_prev.index(i) for i in common_labels]  # Get labels indexes of common detected keypoints from previous frame
                    common_label_idx_curr = [detected_labels.index(i) for i in common_labels]  # Get labels indexes of common detected keypoints from current frame
                    coor_common_label_prev = detected_labels_src_pts_prev[common_label_idx_prev]  # Get labels coordinates of common detected keypoints from previous frame
                    coor_common_label_curr = detected_labels_src_pts[common_label_idx_curr]  # Get labels coordinates of common detected keypoints from current frame
                    coor_error = mean_squared_error(coor_common_label_prev, coor_common_label_curr)  # Calculate error between previous and current common keypoints coordinates
                    update_homography = coor_error > keypoints_displacement_mean_tol  # Check if error surpassed the predefined tolerance level
                else:
                    update_homography = True
            else:
                update_homography = True

            if update_homography:
                h, mask = cv2.findHomography(detected_labels_src_pts, detected_labels_dst_pts)  # Calculate homography matrix

            detected_labels_prev = detected_labels.copy()  # Save current detected keypoint labels for next frame
            detected_labels_src_pts_prev = detected_labels_src_pts.copy()  # Save current detected keypoint coordinates for next frame

            bboxes_p_c_0 = bboxes_p_c[[i == 0 for i in labels_p], :]  # Get bounding boxes information (x,y,w,h) of detected players (label 0)
            bboxes_p_c_2 = bboxes_p_c[[i == 2 for i in labels_p], :]  # Get bounding boxes information (x,y,w,h) of detected ball(s) (label 2)

            # Get coordinates of detected players on frame (x_center, y_center+h/2)
            detected_ppos_src_pts = bboxes_p_c_0[:, :2] + np.array([[0] * bboxes_p_c_0.shape[0], bboxes_p_c_0[:, 3] / 2]).transpose()
            # Get coordinates of the first detected ball (x_center, y_center)
            detected_ball_src_pos = bboxes_p_c_2[0, :2] if bboxes_p_c_2.shape[0] > 0 else None

            # Transform players coordinates from frame plane to tactical map plane using the calculated Homography matrix
            pred_dst_pts = []  # Initialize players tactical map coordinates list
            for pt in detected_ppos_src_pts:  # Loop over players frame coordinates
                pt = np.append(np.array(pt), np.array([1]), axis=0)  # Convert to homogeneous coordinates
                dest_point = np.matmul(h, np.transpose(pt))  # Apply homography transformation
                dest_point = dest_point / dest_point[2]  # Revert to 2D-coordinates
                pred_dst_pts.append(list(np.transpose(dest_point)[:2]))  # Update players tactical map coordinates list
            pred_dst_pts = np.array(pred_dst_pts)

            # Transform ball coordinates from frame plane to tactical map plane using the calculated Homography matrix
            if detected_ball_src_pos is not None:
                pt = np.append(np.array(detected_ball_src_pos), np.array([1]), axis=0)
                dest_point = np.matmul(h, np.transpose(pt))
                dest_point = dest_point / dest_point[2]
                detected_ball_dst_pos = np.transpose(dest_point)

                bola_x = int(detected_ball_dst_pos[0])
                bola_y = int(detected_ball_dst_pos[1])

                # Update track ball position history
                if len(ball_track_history['src']) > 0:
                    if np.linalg.norm(detected_ball_src_pos - ball_track_history['src'][-1]) < ball_track_dist_thresh:
                        ball_track_history['src'].append((int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1])))
                        ball_track_history['dst'].append((int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1])))
                    else:
                        ball_track_history['src'] = [(int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1]))]
                        ball_track_history['dst'] = [(int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1]))]
                else:
                    ball_track_history['src'].append((int(detected_ball_src_pos[0]), int(detected_ball_src_pos[1])))
                    ball_track_history['dst'].append((int(detected_ball_dst_pos[0]), int(detected_ball_dst_pos[1])))

            # Remove oldest tracked ball position if track exceeds threshold
            if len(ball_track_history) > max_track_length:
                ball_track_history['src'].pop(0)
                ball_track_history['dst'].pop(0)

        ######### Part 2 ##########
        # Players Team Prediction #
        ###########################

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert frame to RGB
        obj_palette_list = []  # Initialize players color palette list
        palette_interval = (0, 5)  # Color interval to extract from dominant colors palette (1st to 5th color)
        annotated_frame = frame  # Create annotated frame

        encontrou_jogador_com_bola = False
        jogador_com_bola = False
        min_dist_jogador_bola = 100000000

        ## Loop over detected players (label 0) and extract dominant colors palette based on defined interval
        for i, j in enumerate(list(results_players[0].boxes.cls.cpu().numpy())):
            if int(j) == 0:
                bbox = results_players[0].boxes.xyxy.cpu().numpy()[i, :]  # Get bbox info (x,y,x,y)
                obj_img = frame_rgb[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]  # Crop bbox out of the frame
                obj_img_w, obj_img_h = obj_img.shape[1], obj_img.shape[0]
                center_filter_x1 = np.max([(obj_img_w // 2) - (obj_img_w // 5), 1])
                center_filter_x2 = (obj_img_w // 2) + (obj_img_w // 5)
                center_filter_y1 = np.max([(obj_img_h // 3) - (obj_img_h // 5), 1])
                center_filter_y2 = (obj_img_h // 3) + (obj_img_h // 5)
                center_filter = obj_img[center_filter_y1:center_filter_y2,
                               center_filter_x1:center_filter_x2]
                obj_pil_img = Image.fromarray(np.uint8(center_filter))  # Convert to pillow image

                reduced = obj_pil_img.convert("P", palette=Image.Palette.WEB)  # Convert to web palette (216 colors)
                palette = reduced.getpalette()  # Get palette as [r,g,b,r,g,b,...]
                palette = [palette[3 * n:3 * n + 3] for n in range(256)]  # Group 3 by 3 = [[r,g,b],[r,g,b],...]
                color_count = [(n, palette[m]) for n, m in reduced.getcolors()]  # Create list of palette colors with their frequency
                RGB_df = pd.DataFrame(color_count, columns=['cnt', 'RGB']).sort_values(by='cnt', ascending=False).iloc[palette_interval[0]:palette_interval[1], :]
                palette = list(RGB_df.RGB)  # Convert palette to list (for faster processing)
                annotated_frame = cv2.rectangle(annotated_frame,  # Add center filter bbox annotations
                                                (int(bbox[0]) + center_filter_x1,
                                                 int(bbox[1]) + center_filter_y1),
                                                (int(bbox[0]) + center_filter_x2,
                                                 int(bbox[1]) + center_filter_y2), (0, 0, 0), 2)

                # Update detected players color palette list
                obj_palette_list.append(palette)

        ## Calculate distances between each color from every detected player color palette and the predefined teams colors
        players_distance_features = []
        # Loop over detected players extracted color palettes
        for palette in obj_palette_list:
            palette_distance = []
            palette_lab = [skimage.color.rgb2lab([i / 255 for i in color]) for color in palette]  # Convert colors to L*a*b* space
            # Loop over colors in palette
            for color in palette_lab:
                distance_list = []
                # Loop over predefined list of teams colors
                for c in color_list_lab:
                    distance = skimage.color.deltaE_cie76(color, c)  # Calculate Euclidean distance in Lab color space
                    distance_list.append(distance)  # Update distance list for current color
                palette_distance.append(distance_list)  # Update distance list for current palette
            players_distance_features.append(palette_distance)  # Update distance features list

        ## Predict detected players teams based on distance features
        players_teams_list = []
        # Loop over players distance features
        for distance_feats in players_distance_features:
            vote_list = []
            # Loop over distances for each color
            for dist_list in distance_feats:
                team_idx = dist_list.index(min(dist_list)) // nbr_team_colors  # Assign team index for current color based on min distance
                vote_list.append(team_idx)  # Update vote voting list with current color team prediction
            players_teams_list.append(max(vote_list, key=vote_list.count))  # Predict current player team by vote counting

        #################### Part 3 #####################
        # Updated Frame & Tactical Map With Annotations #
        #################################################

        ball_color_bgr = (0, 0, 255)  # Color (GBR) for ball annotation on tactical map
        j = 0  # Initializing counter of detected players
        palette_box_size = 10  # Set color box size in pixels (for display)

        # Loop over all detected object by players detection model
        for i in range(bboxes_p.shape[0]):
            conf = confs_p[i]  # Get confidence of current detected object
            if labels_p[i] == 0:  # Display annotation for detected players (label 0)

                # Display extracted color palette for each detected player
                palette = obj_palette_list[j]  # Get color palette of the detected player
                for k, c in enumerate(palette):
                    c_bgr = c[::-1]  # Convert color to BGR
                    annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_p[i, 2]) + 3,  # Add color palette annotation on frame
                                                                      int(bboxes_p[i, 1]) + k * palette_box_size),
                                                     (int(bboxes_p[i, 2]) + palette_box_size,
                                                      int(bboxes_p[i, 1]) + (palette_box_size) * (k + 1)),
                                                     c_bgr, -1)

                team_name = list(colors_dic.keys())[players_teams_list[j]]  # Get detected player team prediction
                color_rgb = colors_dic[team_name][0]  # Get detected player team color
                color_bgr = color_rgb[::-1]  # Convert color to bgr

                annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_p[i, 0]), int(bboxes_p[i, 1])),  # Add bbox annotations with team colors
                                                (int(bboxes_p[i, 2]), int(bboxes_p[i, 3])), color_bgr, 1)

                # Add tactical map player position color coded annotation if more than 3 field keypoints are detected
                if len(detected_labels_src_pts) > 3:

                    jogador_j_x = int(pred_dst_pts[j][0])
                    jogador_j_y = int(pred_dst_pts[j][1])

                    cv2.putText(annotated_frame, team_name + f" {j} - {jogador_j_x}, {jogador_j_y}",  # Add team name annotations
                                (int(bboxes_p[i, 0]), int(bboxes_p[i, 1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                color_bgr, 2)

                    # Logic to identify who has the ball
                    dist_euclideana = sci_dist.euclidean((bola_x, bola_y), (jogador_j_x, jogador_j_y))
                    if dist_euclideana < min_dist_jogador_bola:
                        min_dist_jogador_bola = dist_euclideana
                        equipa_com_bola = team_name

                    tac_map_copy = cv2.circle(tac_map_copy, (int(pred_dst_pts[j][0]), int(pred_dst_pts[j][1])),
                                              radius=5, color=color_bgr, thickness=-1)

                j += 1  # Update players counter
            
            else:  # Display annotation for other detections (label 1, 2)
                annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_p[i, 0]), int(bboxes_p[i, 1])),  # Add white colored bbox annotations
                                                 (int(bboxes_p[i, 2]), int(bboxes_p[i, 3])), (255, 255, 255), 1)

                # Only add white colored label text annotations if detected_ball_dst_pos is defined
                if 'detected_ball_dst_pos' in locals() and detected_ball_dst_pos is not None:
                    cv2.putText(annotated_frame, labels_dic[labels_p[i]] + f" {conf:.2f} - {int(detected_ball_dst_pos[0])},{int(detected_ball_dst_pos[1])}", 
                                (int(bboxes_p[i, 0]), int(bboxes_p[i, 1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                                (255, 255, 255), 2)

                # Add tactical map ball position annotation if detected
                if 'detected_ball_dst_pos' in locals() and detected_ball_dst_pos is not None:
                    tac_map_copy = cv2.circle(tac_map_copy, (int(detected_ball_dst_pos[0]),
                                                             int(detected_ball_dst_pos[1])), radius=5,
                                              color=ball_color_bgr, thickness=3)


        # Add goal annotations
        for i in range(bboxes_g.shape[0]):
            goal_bbox = bboxes_g[i]
            conf = confs_g[i]

            # Check if the ball is close to the goal bounding box (goal try)

            # Check if detected_ball_dst_pos is defined before using it
            if 'detected_ball_dst_pos' in locals() and detected_ball_dst_pos is not None:
                # Define the goal area center and radius
                goal_center_x = (goal_bbox[0] + goal_bbox[2]) / 2
                goal_center_y = (goal_bbox[1] + goal_bbox[3]) / 2
                goal_radius = goal_try_threshold  # Use the defined goal_try_threshold as the radius

                # Calculate the distance between the ball and the goal center
                dist_to_goal_center = np.linalg.norm(np.array([goal_center_x, goal_center_y]) - detected_ball_dst_pos[:2])

                # Determine if the ball is heading towards the goal area
                if len(ball_track_history['dst']) > 1:
                    # Get the last two positions of the ball
                    prev_ball_pos = ball_track_history['dst'][-2]
                    curr_ball_pos = ball_track_history['dst'][-1]

                    # Calculate the direction vector of the ball
                    direction_vector = np.array(curr_ball_pos) - np.array(prev_ball_pos)
                    direction_vector = direction_vector / np.linalg.norm(direction_vector)  # Normalize the direction vector

                    # Calculate the vector from the ball to the goal center
                    vector_to_goal_center = np.array([goal_center_x, goal_center_y]) - np.array(curr_ball_pos)
                    vector_to_goal_center = vector_to_goal_center / np.linalg.norm(vector_to_goal_center)  # Normalize the vector to the goal center

                    # Calculate the dot product to determine if the ball is heading towards the goal
                    dot_product = np.dot(direction_vector, vector_to_goal_center)
                    if dot_product > 0.4:  # Adjust the threshold as needed
                        goal_try = True
                    else:
                        goal_try = False
                else:
                    goal_try = False

            # Draw the goal bounding box with the appropriate color
            goal_color = (0, 255, 0) if not goal_try else (0, 0, 255)
            annotated_frame = cv2.rectangle(annotated_frame, (int(goal_bbox[0]), int(goal_bbox[1])),
                                            (int(goal_bbox[2]), int(goal_bbox[3])), goal_color, 2)
            cv2.putText(annotated_frame, f"Goal {conf:.2f}", (int(goal_bbox[0]), int(goal_bbox[1]) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, goal_color, 2)

            # Add "Goal Try" text if goal_try is True
            if goal_try:
                cv2.putText(annotated_frame, "Goal Try", (int(goal_bbox[0]), int(goal_bbox[1]) - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

            # Create a mask for the shadow
            shadow_mask = np.zeros_like(annotated_frame, dtype=np.uint8)

            # Draw filled planes from each corner of the goal bounding box to the nearest keypoints on the mask
            shadow_color = (128, 128, 128)  # Grey for shadow
            for corner in [(goal_bbox[0], goal_bbox[1]), (goal_bbox[2], goal_bbox[1]), 
                           (goal_bbox[0], goal_bbox[3]), (goal_bbox[2], goal_bbox[3])]:
                nearest_kp1, nearest_kp2 = find_two_nearest_keypoints(np.array(corner), detected_labels_src_pts)
                polygon_points = np.array([corner, nearest_kp1, nearest_kp2, corner], dtype=np.int32)
                cv2.fillPoly(shadow_mask, [polygon_points], shadow_color)

            # Blend the shadow mask with the original frame to add transparency
            alpha = 0.5  # Transparency factor
            annotated_frame = cv2.addWeighted(annotated_frame, 1, shadow_mask, alpha, 0)

            # Check if the ball is within the goal bounding box
            if detected_ball_dst_pos is not None:
                if (goal_bbox[0] <= detected_ball_dst_pos[0] <= goal_bbox[2]) and (goal_bbox[1] <= detected_ball_dst_pos[1] <= goal_bbox[3]):
                    goal = True
                else:
                    goal = False

        # Logic to identify the current action (pass, dribble, or interception)
        if min_dist_jogador_bola < threshold_posse_bola:
            jogador_com_bola = True
            passe = False
            interceccao = False

            # Heuristic 3: Reception: if the last action was a pass and the current team is the same as the previous one
            recepcao = equipa_anterior_com_bola == equipa_com_bola
        else:
            # Heuristic 1: Pass is when the same team keeps ball possession
            passe = equipa_com_bola == equipa_anterior_com_bola
            # Heuristic 2: Interception is when the ball is controlled by the opposing team
            interceccao = equipa_com_bola != equipa_anterior_com_bola
            jogador_com_bola = False

        equipa_anterior_com_bola = equipa_com_bola

        for i in range(bboxes_k.shape[0]):
            annotated_frame = cv2.rectangle(annotated_frame, (int(bboxes_k[i, 0]), int(bboxes_k[i, 1])),  # Add bbox annotations with team colors
                                            (int(bboxes_k[i, 2]), int(bboxes_k[i, 3])), (0, 0, 0), 1)

        # Plot the ball tracks on tactical map
        if len(ball_track_history['src']) > 0:
            points = np.hstack(ball_track_history['dst']).astype(np.int32).reshape((-1, 1, 2))
            tac_map_copy = cv2.polylines(tac_map_copy, [points], isClosed=False, color=(0, 0, 100), thickness=2)

        # Add the arrow representing ball direction
        if len(ball_track_history['dst']) > 1:
            # Use the last two positions to determine the direction
            start_point = ball_track_history['dst'][-2]
            end_point = ball_track_history['dst'][-1]
            # Calculate the direction vector
            direction_vector = np.array(end_point) - np.array(start_point)
            # Predict the future position
            predicted_end_point = np.array(end_point) + direction_vector
            # Draw the arrow
            tac_map_copy = cv2.arrowedLine(tac_map_copy, tuple(end_point), tuple(predicted_end_point.astype(int)), (0, 255, 0), 3)

        # Combine annotated frame and tactical map in one image with colored border separation
        border_color = [255, 255, 255]  # Set border color (BGR)
        annotated_frame = cv2.copyMakeBorder(annotated_frame, 40, 10, 10, 10,  # Add borders to annotated frame
                                             cv2.BORDER_CONSTANT, value=border_color)
        tac_map_copy = cv2.copyMakeBorder(tac_map_copy, 70, 50, 10, 10, cv2.BORDER_CONSTANT,  # Add borders to tactical map
                                          value=border_color)
        tac_map_copy = cv2.resize(tac_map_copy, (tac_map_copy.shape[1], annotated_frame.shape[0]))  # Resize tactical map
        final_img = cv2.hconcat((annotated_frame, tac_map_copy))  # Concatenate both images

        ## Add info annotation
        cv2.putText(final_img, f"Tactical Map {frame_nbr}", (1370, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)
        cv2.putText(final_img, f"Equipa com bola: {equipa_com_bola}", (1320, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(final_img, f"Dist: {int(min_dist_jogador_bola)}", (1320, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        # Print the current action
        cv2.putText(final_img, f"Passe" if passe else "", (1430, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(final_img, f"Interceccao" if interceccao else "", (1430, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(final_img, f"Drible" if jogador_com_bola else "", (1530, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(final_img, f"Goal Try" if goal_try else "", (1630, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(final_img, f"Goal!" if goal else "", (1730, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        cv2.putText(final_img, "Press 'p' to pause & 'q' to quit", (820, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)

        new_frame_time = time.time()  # Get time after finished processing current frame
        fps = 1 / (new_frame_time - prev_frame_time)  # Calculate FPS as 1/(frame processing duration)
        prev_frame_time = new_frame_time  # Save current time to be used in next frame
        cv2.putText(final_img, "FPS: " + str(int(fps)), (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 0), 2)

        # Initialize the video writer after obtaining final_img dimensions
        if out is None:
            height, width = final_img.shape[:2]
            out = cv2.VideoWriter(output_path, fourcc, frame_rate, (width, height))

        # Write the processed frame to the video file
        out.write(final_img)

        # Display the final annotated frame
        plt.imshow(final_img)
        plt.show()

        # Treat keyboard user inputs ("p" for pause/unpause & "q" for quit)
       
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and video writer
cap.release()
if out:
    out.release()


YOLOv5 🚀 2024-7-18 Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 14918MiB)



requirements: /opt/conda/lib/python3.10/site-packages/yolov5/requirements.txt not found, check failed.


Fusing layers... 
Model summary: 214 layers, 7022326 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 



0: 384x640 21 players, 2 referees, 70.7ms
Speed: 8.0ms preprocess, 70.7ms inference, 221.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TR6MC, 1 TL6MC, 1 TR6ML, 1 TL6ML, 1 TR18MC, 1 TL18MC, 1 TR18ML, 1 TL18ML, 1 TRArc, 1 TLArc, 1 RMC, 1 LMC, 1 LML, 1 BR6MC, 1 BL6MC, 1 BL18MC, 1 BRArc, 1 BLArc, 60.7ms
Speed: 1.9ms preprocess, 60.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 players, 2 referees, 1 ball, 39.5ms
Speed: 2.3ms preprocess, 39.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TR6MC, 1 TL6MC, 1 TL18MC, 1 TRArc, 1 TLArc, 1 RMC, 1 LMC, 1 LML, 1 BR6MC, 1 BL6MC, 1 BL18MC, 1 BL18ML, 1 BRArc, 1 BLArc, 25.1ms
Speed: 1.4ms preprocess, 25.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 players, 1 referee, 39.5ms
Speed: 1.6ms preprocess, 39.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 TR6MC, 1 TL6MC, 1 TL18MC, 1 TRArc, 1 TLAr